# CHP 5 . Support Vector Machines

- __ 발표자 : 정지원 __
- __ 발표일 : 2017. 7. 2(일)__

서포트 벡터 머신은 매우 강력하고 유용한 머신러닝 모델이다. 선형 혹은 비선형 분류/회귀 모델 또는 이상치 감지에도 사용되곤 한다. 가장 인기 있는 머신러닝 모델 중 하나이며,복잡하지만 작거나 중간 사이즈의 데이터 셋의 분류모델에 특별히 적합하다고 알려져 있다. 이번 장에서는 SVM의 핵심 개념에 대해 다루고, 어떻게 작동하고 어떻게 사용하는지 알아본다.

## Linear SVM Classification

![Image](figures/1.png)

__어떻게 분리할 것인가? -> 큰 margin을 갖도록 분리하자.__

![Image](figures/2.png)

SVM같은 경우 스케일에 민감하다. 반드시 표준화를 해줘야 한다.

## Soft Margin Classification

위에서 다룬 것과 같이, 반드시 2개의 영역이 잘 분류 되는 경우를 Hard Margin Classification이라고 한다. 그러나 여기엔 두 가지 문제가 있다. 첫 번째는 데이터가 선형 분리가 되는 경우에만 작동한다는 것이다. 두 번째는 이상치(outlier)에 매우 민감하다. 그림을 살펴보자.

![Image](figures/3.png)

이러한 문제를 없애고 좀 더 유연한 모델을 만들 필요가 있다. 목표는 가능한 거리를 길게 유지하는 것과 마진 위반(거리의 중간이나 잘못된 쪽에서 끝나는 경우)을 제한하는 것 사이의 균형을 찾는 것이다. 이를 soft margin 분류라 한다.

Scikit-learn의 SVM에서는, _**C**_ 하이퍼파라미터를 통해 조절할 수 있다.

![Image](figures/4.png)



In [4]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris-Virginica
svm_clf = Pipeline((("scaler", StandardScaler()),("linear_svc", LinearSVC(C=1, loss="hinge")),))
svm_clf.fit(X, y)

Pipeline(steps=(('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))))

In [5]:
svm_clf.predict([[5.5, 1.7]])

array([ 1.])

Logistic Regression과는 다르게, SVM은 Class에 대한 확률값은 제공하지 않는다.

대안으로, SVM class에 있는 SVC(kernel="linear", C=1)를 사용할 수 있다. 큰 데이터 셋에서는 느린 편이기 때문에 추천되지는 않는다. 다른 옵션으로는 SGDClassifier가 있다. Linear SVC보다는 빨리 수렴하지는 않지만, 아주 큰 데이터셋을 다루기엔 좋다.(Stochastic 방식이라서) 혹은 online 학습이 가능하다.

## Nonlinear SVM Classification

비록 SVM 분류기가 효율적이고 많은 경우에 좋은 결과를 보이지만, 대부분의 데이터는 linearly separable하지 않다. 이러한 비선형 데이터를 다루는 방법은, polynomial feature와 같은 다른 features를 추가하는 것이다.

![Image](figures/5.png)

In [8]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
polynomial_svm_clf = Pipeline((
            ("poly_features", PolynomialFeatures(degree=3)),
            ("scaler", StandardScaler()),
            ("svm_clf", LinearSVC(C=10, loss="hinge"))
        ))
polynomial_svm_clf.fit(X, y)

Pipeline(steps=(('poly_features', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))))

![Image](figures/6.png)

## Polynomial Kernel

polynomial feature를 추가하는 것은 SVM뿐만 아니라 여러 머신러닝 알고리즘의 결과를 좋게 만들 수 있는 간단한 방법이다. 낮은 polynomial 차원으로는 아주 복잡한 데이터셋은 다루기 힘들고, 고차원의 polynomial 차원으로는 아주 큰 수의 feature 수를 만들어내기 때문에 모델을 느리게 만든다.

운좋게, SVM의 kernel trick이라는 기적적인 수학 테크닉을 사용할 수 있다. 실제로 다항식을 추가할 필요없이 많은 다항식을 추가한 것과 같은 결과를 얻을 수 있다. 따라서 실제로 어떤 기능도 추가하지 않기 때문에 feature 수의 조합 폭발이 없다. 이 트릭은 SVC 클래스에 의해 구현된다. moon 데이터 세트에서 테스트 해보자.

In [10]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
        ))
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=(('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))))

d는 degree를 뜻한다. 여기서 coef0(r)값은 얼마나 모델이 고차원의 polynomail과 저차원의 polynomial에 얼마나 영향을 받을지의 정도를 조절해준다.

## Adding Similarity Features

비선형 문제를 해결할 또 다른 테크닉은 유사도 함수를 통해 계산하여 새로운 feature를 추가하는 것이다. 이것은 각 인스턴스 들이 특정 _landmark_와 얼마나 잘 닮았는지의 정도를 측정한다. 예를 들어, 1차원의 데이터셋에 대해 얘기해보자. 여기에 두개의 landmarks인 x1 = -2, x1 = 1을 추가해보자. 그러고서는 유사도 함수를 __Gaussian Radial Basis Function(RBF)__을 통해 구해본다.

![Image](figures/8.png)
![Image](figures/9.png)

아마 Landmarks를 어떻게 선택하는지 궁금할 것이다. 가장 쉬운 방법은 각각의 인스턴스의 위치에 모두 두는 방법이다. 이렇게 되면 아주 많은 차원이 될 것이고, linearly separable하게 나눠질 기회가 많을 것이다. 단점은 n개의 feature와 m개의 instance를 갖는 데이터가 m개의 feature에 m개의 instance를 갖는 데이터로 바뀐다는 것이다. 만약에 training set이 아주 크다면, 결국 똑같이 많은 수의 feature로 끝나게 된다.

## Gaussian RBF Kernel

Polynomial Feature 방법과 유사하게, Similarity Feature 방법은 모든 머신러닝 알고리즘에서 유용할 수 있지만, 아주 큰 트레이닝 세트에서 모든 추가되는 feature를 계산하는 많은 비용이 소요될 수 있다. 그러나 커널 트릭은 SVM 매직을 다시 한번 수행한다. 실제로 많은 추가 기능을 추가하지 않고도 비슷한 결과를 얻을 수 있다. SVC 클래스를 사용하여 Gaussian RBF 커널을 사용해 보자.

In [12]:
rbf_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
        ))
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=(('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))))

하이퍼 파라미터 감마(γ)와 C의 다른 값으로 훈련 된 모델을 보자. 감마를 증가 시키면 종 모양 커브가 좁아지고, 결과적으로 각 인스턴스의 영향 범위는 더 작아진다. 결정 경계가 더욱 불규칙 해지고 개별 인스턴스 주위를 움직인다. 반대로 감마 값이 작으면 벨 모양의 곡선이 넓어 지므로 인스턴스의 영향 범위가 넓어지고 결정 경계가 더 매끄럽게 된다. 그래서 γ는 정규화 hyperparameter와 같은 역할을 한다. 모델이 오버피팅(overfitting)이라면 그것을 줄여야하며, 피팅이 맞지 않으면 모델을 증가시켜야한다.(C 하이퍼 매개 변수와 유사)

![Image](figures/10.png)

다른 커널들도 있지만 잘 사용되지 않는다. 특정 데이터 구조에 특별하게 좋은 커널들이 있다. _String kernels_은 때로 텍스트 문서나 DNA sequence에 좋기로 알려져있다.

![Image](figures/11.png)

## Computational Complexity

_liblinear_ 라이브러리의 __LinearSVC__ 클래스는 linear SVMs을 최적화 하기 위한 알고리즘을 가지고 있다. 커널 트릭을 지원하지 않으며, 대략 O(m\*n)의 복잡도를 갖는다.

매우 높은 정밀도가 필요한 경우 알고리즘이 오래 걸린다. 이것은 허용 오차 hyperparameter ε (Scikit-Learn에서 _tol_이라고 함)에 의해 조절된다. 대부분의 분류 작업에서 기본 tolerance는 문제 없다.

SVC 클래스는 커널 트릭을 지원하는 알고리즘을 구현하는 libsvm 라이브러리를 기반으로 한다. 트레이닝 시간 복잡도는 일반적으로 O(m^2\*n)와 O (m^3\*n) 사이다. 안타깝게도 이는 교육 인스턴스 수가 많아지면 (ex:수십만 건) 느리게 진행될 수 있음을 의미한다. 이 알고리즘은 복잡하지만 작은 또는 중간 트레이닝 세트에 적합하다. 그러나 feature 수가 많아지며, 특히 Sparse feature에 적용됩니다. 이 경우 알고리즘은 인스턴스 당 평균 0이 아닌 feature의 수로 대략적으로 조정된다. 다음 표는 Scikit-Learn의 SVM 분류 클래스를 비교한다.

![Image](figures/12.png)

## SVM Regression

이전에 언급했듯이 SVM 알고리즘은 선형 및 비선형 분류를 지원할뿐만 아니라 선형 및 비선형 회귀를 지원한다. trick 위반을 제한하면서 두 클래스 사이에 가능한 가장 큰 거리를 맞추려고 시도하는 대신 SVM 회귀는 가능한 많은 인스턴스를 거리에 적용하면서 마진 위반을 제한하려고 시도한다.(ex:거리) 거리의 너비는 하이퍼파라미터 ε에 의해 제어된다. 큰 마진 (ε=1.5)과 작은 마진 (ε=0.5)이 있는 임의의 선형 데이터에 대해 학습 한 두 개의 선형 SVM 회귀 모델을 보자.

![Image](figures/13.png)

마진 안에 training instance를 추가하는 것은 모델의 예측에 영향을 주지 않는다. 이러한 모델을 __ε-insensitive__라고 한다.

In [13]:
from sklearn.svm import LinearSVR
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

비선형 회귀 작업을 처리하기 위해 커널화 된 SVM 모델을 사용할 수 있다. 예를 들어, 다음 그림은 2차 다항 커널을 사용하는 무작위 2차 트레이닝 세트의 SVM 회귀를 보여준다. 왼쪽에는 정규화가 거의없고 오른쪽엔 훨씬 많은 정규화가 있다.

![Image](figures/14.png)

다음 코드는 Scikit-Learn의 SVR 클래스(커널 트릭을 지원)를 사용하여 모델을 생성한다. SVR 클래스는 SVC 클래스의 회귀와 같으며, LinearSVR 클래스는 LinearSVC 클래스의 회귀와 동일하다. LinearSVR 클래스는(LinearSVC클래스와 마찬가지로) 트레이닝 세트의 크기와 선형으로 확장되지만, 트레이닝 세트가 커지면(SVC 클래스와 마찬가지로) SVR 클래스가 아주 느려진다.

In [14]:
from sklearn.svm import SVR
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## Under the Hood

이 섹션에서는 선형 SVM 분류로 시작하여 SVM이 어떻게 예측을 하고 어떻게 트레이닝 알고리즘이 작동하는지 설명한다. 머신러닝을 시작한지 얼마 안됐다면 건너 뛰고 연습 문제로 바로 넘어가라. 나중에 SVM에 대해 더 깊이 이해하고자 할 때 다시 돌아오자.

첫 째, 표기법에 대한 단어 : 제 4 장에서는 바이어스 항 θ0과 입력 특징 가중치 θ1 ~ θn을 포함하여 모든 모델 매개 변수를 하나의 벡터 θ에 넣고 모든 인스턴스에 바이어스 입력 x0 = 1을 추가하는 관례를 사용했었다 . 이 장에서는 SVM을 다룰 때보다 편리한(더 일반적인) 규칙을 사용할 것이다. 바이어스 식을 b라고 부르며 feature weight 벡터를 w라고 한다. 바이어스 feature가 input feature 벡터에 추가되지 않는다.

## Decision Function and Predictions

선형 SVM 분류기 모델은 단순히 결정 함수 wT·x+b = w1\*x1 + ⋯ + wn\*xn + b를 계산하여 새로운 인스턴스 x의 클래스를 예측한다. 결과가 양수이면 예측 클래스 ŷ는 긍정 Class(1), 그렇지 않으면 그것은 부정 Class(0)이다.

![Image](figures/15.png)

Data set은 꽃잎 폭과 꽃잎 길이라는 두 가지 feature가 있기 때문에 이차원 평면이다. 결정 경계는 결정 함수가 0 인 지점 집합이다. 즉 두 평면의 교차점이다.(두꺼운 실선)

![Image](figures/16.png)

파선은 결정 함수가 1 또는 -1 인 점을 나타내며, 평행하고 결정 경계와 동일한 거리에 있으며 그 주위에 여백을 형성한다.

선형 SVM 분류자를 훈련한다는 것은 마진 위반(hard 마진)을 피하거나 한계(소프트 마진)를 제한하면서 이 여백을 최대한 넓게 만드는 w 및 b의 값을 찾는 것을 의미한다.

## Training Objective

결정 함수의 기울기를 고려해보자. 이것은 가중치 벡터의 표준인 ∥w∥와 같다. 이 기울기를 2로 나눈 경우, 결정 함수가 ±1인 점은 결정경계에서 두 배 멀리 떨어진다. 즉, 기울기를 2로 나누면 마진에 2가 곱해진다. 가중치 벡터 w가 작을수록 마진이 커진다.

![Image](figures/17.png)


따라서 ∥w∥를 최소화하여 큰 마진을 얻어야 한다. 그러나 마진 위반 (hard 마진)을 피하려면 모든 긍정적인 training 인스턴스에 대해 1보다 커야하고 부정적인 training 인스턴스의 경우에는 -1보다 작아야한다. 양의 인스턴스 (y(i)=1 인 경우)에 대해 음의 인스턴스 (y(i)=0) 및 t(i)=1에 대해 t(i)=-1을 정의하면이 제약 조건을 t(i)(wT · x(i) + b) ≥ 1이다.

그러므로 우리는 하드 마진 선형 SVM 분류기 목적을 다음 Constrained 최적화 문제로 표현할 수 있다.

![Image](figures/18.png)

![Image](figures/19.png)


소프트 마진 목표를 얻으려면 각 인스턴스에 대해 여유 변수 ζ(i ≥ 0을 도입해야 한다. ζ(i)는 i번째 인스턴스가 마진을 위반하는 정도를 측정한다. 우리는 이제 두 가지 상충되는 목표를 가지고 있다.

마진 위반을 줄이고 마진을 높이기 위해 가능한 한 작게 1/2 \* wT · w로 만든다. 이것이 C 하이퍼 파라미터가 들어있는 지점이다. 이 두 목표 사이의 절충점을 정의 할 수 있다.

![Image](figures/20.png)

## Quadratic Programming

하드 마진과 소프트 마진 문제는 모두 선형 제약 조건에 대한 볼록한 2차 최적화 문제다. 이러한 문제를 2차 프로그래밍(QP) 문제라고 한다. 많은 솔버가 이 책의 범위를 벗어나는 다양한 기술을 사용하여 QP 문제를 해결할 수 있다.

![Image](figures/21.png)

![Image](figures/22.png)

따라서 하드 마진 선형 SVM 분류기를 훈련하는 한 가지 방법은 이전 매개 변수를 전달하여 기성품의 QP 솔버를 사용하는 것이다. 결과 벡터 p는 i=1, 2, ⋯, m에 대해 바이어스 항 b = p0 및 특징 가중치 wi = pi를 포함한다. 마찬가지로 QP 솔버를 사용하여 소프트 마진 문제를 해결할 수 있다.

그러나 커널 트릭을 사용하기 위해 다른 제약이 있는 최적화 문제를 살펴볼 것이다.

## The Dual Problem

최적화 문제가 주어지면 다른 문제이지만 밀접한 관련이 있는 다른 문제를 표현할 수 있다.(Dual 문제) Dual 문제에 대한 해법은 일반적으로 근원 문제의 해법에 대한 경계를 낮추지만, 일부 조건 하에서는 심지어 문제와 동일한 해를 가질 수 있다. 다행히도 SVM 문제는 이러한 조건을 충족 시키므로 원래 문제 또는 Dual 문제를 해결할 수 있다. 두 가지 모두 동일한 솔루션을 갖게된다.

![Image](figures/23.png)

이 방정식을 최소화하는 벡터α(QP 해석기 사용)를 찾으면 다음 방정식을 사용하여 원래 문제를 최소화하고 계산할 수 있다.

![Image](figures/24.png)

트레이닝 인스턴스의 수가 feature의 수보다 작은 경우, 이중 문제는 primal보다 더 빨리 풀 수 있다. 더 중요한 것은, 원래 불가능 했던 커널 트릭을 가능하게 만들기도 한다. 이 커널 트릭은 도대체 무엇인가?

## Kernelized SVM

2차원 다항식 변환을 2차원 트레이닝 세트(ex:Moon 트레이닝 세트)에 적용한 다음 변형 된 트레이닝 세트에서 선형 SVM 분류기를 학습한다고 가정한다. 다음은 적용하려는 2차 다항식 매핑 함수 φ를 보여준다.

![Image](figures/25.png)

변환된 벡터는 2차원이 아닌 3차원임을 유의하라. 이제 이 2차 다항식 매핑을 적용하고 변환된 벡터의 내적을 계산하면 2차원 벡터 2개에 어떤 일이 발생하는지 살펴보자.

![Image](figures/26.png)

여기에 핵심이 있다. 만약 당신이 모든 훈련 과정에 변환 φ를 적용한다면, Dual 문제는 내적 (φ)을 포함할 것이다. 그러나 φ가 식 5-8에서 정의 된 2차 다항식 변환이라면, 이 변환 된 벡터의 내적을 $(X^{(i)^T} · X^{(j)})^2$로 간단하게 대체 할 수 있다.

따라서 실제로는 트레이닝 인스턴스를 전혀 변형 할 필요가 없다. 방정식의 내적을 사각형으로 바꾼다. 결과는 training set을 실제로 변환한 후 선형 SVM 알고리즘에 맞추는 문제를 겪었을 때와 똑같다. 그러나 이 트릭은 전체 프로세스를 훨씬 더 효율적으로 계산한다. 이것이 커널 트릭의 본질이다.

함수 $K(a, b) = (a^T · b)^2$는 2차 다항 커널로 불린다. 기계 학습에서 커널은 변환 φ를 계산할 필요 없이(또는 심지어 알기 위해) 원본 벡터 a와 b에만 기초하여 내적을 계산할 수 있는 함수다. 다음은 가장 일반적으로 사용되는 커널을 나열한다.

![Image](figures/27.png)

![Image](figures/28.png)

우리가 묶어야 할 하나의 느슨한 끝이 아직 있다. 방정식 5-7은 선형 SVM 분류기의 경우 Dual 해에서 근원 해법으로 가는 방법을 보여준다. 그러나 커널 트릭을 적용하면 φ(x(i))가 포함 된 방정식이 된다. 사실 φ(x(i))와 같은 차원 수를 가져야 한다. 거대한 크기 또는 무한 크기일 수 있으므로 계산할 수가 없다. 그러나 어떻게 알지 못하면서 예측을 할 수 있을까? 음, 좋은 소식은 방정식 5-7의 공식을 새로운 인스턴스 x(n)의 결정 함수에 넣을 수 있고 입력 벡터 사이에 dot product만 있는 방정식을 얻을 수 있다는 것이다. 이렇게하면 커널 트릭을 다시 사용할 수 있다.

![Image](figures/29.png)

Support 벡터에 대해서만 α(i) ≠ 0이므로, 예측을 하는 것은 모든 학습 인스턴스가 아닌 지원 벡터만으로 새로운 입력 벡터 x(n)의 내적을 계산하는 것을 의미한다. 물론, 같은 트릭을 사용하여 바이어스 항 b를 계산해야 한다.

![Image](figures/30.png)

두통을 겪기 시작하면 완벽하게 정상이다. 불행히도 커널 트릭의 부작용이다.

## Online SVMs

이 장을 끝내기 전에 온라인 SVM 분류 자에 대해 간략히 살펴보도록 하자. 온라인 학습은 일반적으로 새 인스턴스가 도착할 때 점진적으로 학습한다는 것을 의미한다. 선형 SVM 분류기에 대한 하나의 방법은 원래 문제(primal problem)로부터 도출 비용 함수를 최소화하기 위해 Gradient Descent(예를 들어, SGDClassifier 사용)를 사용하는 것이다. 불행히도 QP를 기반으로 하는 방법보다 훨씬 느리게 수렴한다.

![Image](figures/31.png)

비용 함수의 첫 번째 합계는 모델이 작은 가중치 벡터 w를 갖도록 밀어 넣어 큰 여백을 만든다. 두 번째 합계는 모든 마진 위반의 합계를 계산한다. 거리에서 벗어나 올바른 위치에 있는 경우 인스턴스의 여백 위반은 0과 같거나 그렇지 않으면 거리의 올바른 면까지의 거리에 비례한다. 이 식을 최소화하면 모델이 마진 위반을 가능한 적게 만든다.

![Image](figures/32.png)

![Image](figures/33.png)

예를 들어, "증가 및 감소 SVM 학습" 또는 "온라인 및 능동 학습을 통한 고속 커널 분류기"를 사용하여 온라인 커널화 된 SVM을 구현할 수도 있다. 그러나 이들은 Matlab 및 C++에서 구현된다. 대규모 비선형 문제의 경우 신경망을 대신 사용하는 것이 좋다.